<a href="https://colab.research.google.com/github/dhriman-deka/Dream-11-Team-Creator/blob/main/%F0%9F%A7%A0_Your_24_7_Mental_Health_Companion_Meet_the_AI_Built_with_Purpose_%F0%9F%A4%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧠 Mental Health Counseling AI Assistant: Fine-Tuning DeepSeek R1 8B
> "The best way to predict the future is to create it." - Abraham Lincoln
Welcome to this specialized notebook for creating a mental health counseling AI assistant! We'll transform the powerful DeepSeek R1 8B reasoning model into a supportive, empathetic counseling assistant using the Unsloth library for blazingly fast fine-tuning.
What makes this project special:
Focuses on mental health support - a critical need in today's world
Uses DeepSeek R1 8B - specifically designed for strong reasoning capabilities
Leverages Unsloth for 2x faster training and lower memory usage
Creates an ethical AI counseling tool following best practices
Let's build something meaningful together! 💪
1. 🚀 Setting Up Our Environment
First, we need to install all the necessary libraries. This step is like gathering our tools before building a house - essential for a solid foundation!
The Unsloth library will be our secret weapon, making training up to 2x faster while using significantly less memory. This means we can train more effectively even with limited GPU resources.


In [4]:
%%capture
import os
# Install required packages
!pip install unsloth
!pip install transformers accelerate bitsandbytes peft trl datasets

2. 🤖 Loading Our Base Model: DeepSeek R1 8B
Now we're choosing our AI "brain" - the DeepSeek R1 8B model. This model is known for its exceptional reasoning capabilities, making it perfect for understanding the nuances of human emotions and mental health concerns.
We'll enhance the model using LoRA (Low-Rank Adaptation), a brilliant technique that allows us to fine-tune just a small portion of the model parameters (about 1-3%) while maintaining most of its knowledge. This is like teaching a seasoned therapist a new specialization rather than training someone from scratch!

In [8]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/DeepSeek-R1-Distill-Llama-8B-unsloth-bnb-4bit",
    max_seq_length=max_seq_length,
    load_in_4bit=load_in_4bit,
)

model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
)

print(f"Model loaded successfully: DeepSeek R1 8B with Unsloth optimization")

==((====))==  Unsloth 2025.3.14: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.3.14 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


Model loaded successfully: DeepSeek R1 8B with Unsloth optimization


3. 📊 Preparing Our Mental Health Counseling Dataset
Great counselors learn from experience. Similarly, our AI needs to learn from high-quality counseling conversations.
We're using a specialized dataset containing over 3,500 mental health counseling conversations between individuals seeking help and professional therapists. This dataset is like a treasure trove of wisdom from experienced counselors!
Our custom prompt format will frame these conversations in a way that helps the model understand the counseling context - distinguishing between the person's concern and the professional's response.
> 💡 Did you know? The format of training data significantly impacts how the model responds. Our counseling-specific format helps establish the right tone and approach.

In [9]:
from datasets import load_dataset

# Define our specialized counseling prompt template
counseling_prompt = """Below is a conversation between a person seeking mental health support and a professional counselor. The counselor provides thoughtful, supportive, and ethical guidance.

### Person's Concern:
{}

### Counselor's Response:
{}"""

# Prepare formatting function
EOS_TOKEN = tokenizer.eos_token

def format_counseling_data(examples):
    contexts = examples["Context"]
    responses = examples["Response"]
    texts = []

    for context, response in zip(contexts, responses):
        # Format with our counseling template and add EOS token
        text = counseling_prompt.format(context.strip(), response.strip()) + EOS_TOKEN
        texts.append(text)

    return {"text": texts}

# Load the mental health dataset
print("Loading mental health counseling dataset...")
dataset = load_dataset("Amod/mental_health_counseling_conversations")

# Format the dataset for training
formatted_dataset = dataset.map(
    format_counseling_data,
    batched=True,
    remove_columns=["Context", "Response"]
)

# Get the train split
train_dataset = formatted_dataset["train"]
print(f"Dataset prepared with {len(train_dataset)} examples")

# Display a sample
print("\nSample from formatted dataset:")
print(train_dataset[0]['text'][:500] + "...")

Loading mental health counseling dataset...


Generating train split:   0%|          | 0/3512 [00:00<?, ? examples/s]

Map:   0%|          | 0/3512 [00:00<?, ? examples/s]

Dataset prepared with 3512 examples

Sample from formatted dataset:
Below is a conversation between a person seeking mental health support and a professional counselor. The counselor provides thoughtful, supportive, and ethical guidance.

### Person's Concern:
I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.
   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.
   How can I change my feeling of being w...



4. 🔥 Fine-Tuning Our Mental Health Counselor
This is where the magic happens! We're teaching our AI model to become a supportive, ethical counseling assistant.
We're using carefully selected training parameters specifically optimized for mental health counseling. For example:
A lower learning rate (1e-4) to preserve the model's existing knowledge while carefully adapting it
A cosine learning rate schedule for smoother, more nuanced learning
No packing to ensure responses maintain their coherent structure
> ⚠️ Important Note: Fine-tuning takes time and GPU resources. Be patient - you're creating something valuable!

In [10]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

# Configure training
print("Configuring training...")

# Show initial GPU memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved initially.")

# Set up the trainer with optimized parameters for mental health counseling
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,  # Better for precise counseling responses
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_ratio=0.05,  # Gradual warmup
        max_steps=300,      # Increased for better performance
        learning_rate=1e-4, # Lower learning rate for sensitive domain
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=10,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="cosine", # Cosine scheduler helps with nuanced learning
        seed=3407,
        output_dir="mental_health_counselor_model",
        report_to="none",
    ),
)

# Run training
print("Starting training...")
trainer_stats = trainer.train()

# Display training stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
print(f"Training completed in {round(trainer_stats.metrics['train_runtime']/60, 2)} minutes")
print(f"Peak memory used: {used_memory} GB ({used_percentage}% of available GPU memory)")

Configuring training...
GPU = Tesla T4. Max memory = 14.741 GB.
5.748 GB of memory reserved initially.


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/3512 [00:00<?, ? examples/s]

Starting training...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 3,512 | Num Epochs = 1 | Total steps = 300
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,3.165500
20,2.607900
30,2.290300
40,2.244500
50,2.202500
60,2.172600
70,2.095200
80,2.152100
90,2.087200
100,2.339000


Training completed in 48.17 minutes
Peak memory used: 7.742 GB (52.52% of available GPU memory)


5. 🧪 Testing Our Counseling Assistant
The moment of truth! Let's see how well our AI counselor responds to various mental health scenarios.
We've crafted diverse test cases representing common counseling scenarios:
Depression symptoms
Anxiety and panic attacks
Stress management
Self-harm concerns
Supporting others in distress
The best way to evaluate our model is to see it in action with real-world questions. Pay attention to how empathetic, helpful, and ethically responsible the responses are.
> 💡 Pro Tip: When evaluating responses, look for empathy, practical advice, appropriate boundaries, and recognition of when professional help is needed.

In [11]:
# Prepare model for inference
FastLanguageModel.for_inference(model)

# Sample test cases for mental health counseling
test_questions = [
    "I've been feeling really down lately and having trouble getting out of bed. I've lost interest in things I used to enjoy. Is this just a phase?",
    "I'm having panic attacks almost daily. My heart races, I can't breathe, and I feel like I'm dying. What can I do when this happens?",
    "I'm feeling overwhelmed with work and family responsibilities. I can't sleep well and feel anxious all the time. How can I manage this stress?",
    "I'm struggling with thoughts of self-harm when I feel worthless. I haven't acted on them, but they're scaring me. What should I do?",
    "My friend has been acting differently lately - withdrawn, missing work, and making concerning comments about 'not being around much longer.' How should I approach this?"
]

# Test the model with streaming output
from transformers import TextStreamer

print("\n===== TESTING FINE-TUNED COUNSELING MODEL =====\n")

for i, question in enumerate(test_questions):
    print(f"\n----- Test Case {i+1} -----")
    print(f"Question: {question}\n")
    print("Response:")

    inputs = tokenizer(
        [counseling_prompt.format(question, "")],
        return_tensors="pt"
    ).to("cuda")

    text_streamer = TextStreamer(tokenizer)
    _ = model.generate(
        **inputs,
        streamer=text_streamer,
        max_new_tokens=300,
        temperature=0.7,
        top_p=0.9,
        do_sample=True
    )
    print("\n")


===== TESTING FINE-TUNED COUNSELING MODEL =====


----- Test Case 1 -----
Question: I've been feeling really down lately and having trouble getting out of bed. I've lost interest in things I used to enjoy. Is this just a phase?

Response:
<｜begin▁of▁sentence｜>Below is a conversation between a person seeking mental health support and a professional counselor. The counselor provides thoughtful, supportive, and ethical guidance.

### Person's Concern:
I've been feeling really down lately and having trouble getting out of bed. I've lost interest in things I used to enjoy. Is this just a phase?

### Counselor's Response:
It is hard to know whether or not this is just a phase.  I would suggest that you talk to your doctor about the possibility of depression.  Depression can be treated with medication or counseling.  If you are already on medication, it is also important to talk to your doctor about any changes in your medication, or any other concerns that you may have.  If you are not on m

6. 💾 Saving Your Counseling Assistant
Great work! Now let's save your valuable creation so you can use it anytime.
This step saves both the LoRA adapters (the specialized knowledge we've added) and the tokenizer (which helps the model understand and generate text). Optionally, you can also export to GGUF format for deployment on consumer hardware.
> 💡 Future Use Tip: You can reload this model later using FastLanguageModel.from_pretrained("your_saved_path") and then easily deploy it via API or interface.

In [12]:
# Save the fine-tuned LoRA adapters
output_dir = "mental_health_counselor_deepseek_r1_8b"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
print(f"Model saved to {output_dir}")

# Optional: Export to GGUF for deployment
convert_to_gguf = False  # Set to True when ready to export
if convert_to_gguf:
    print("Converting to GGUF format for deployment...")
    model.save_pretrained_gguf(
        output_dir + "_gguf",
        tokenizer,
        quantization_method="q4_k_m"  # Good balance of quality and size
    )
    print(f"GGUF model saved to {output_dir}_gguf")

Model saved to mental_health_counselor_deepseek_r1_8b


7. 🛡️ Ethical Guidelines and Responsible Use
With great power comes great responsibility. This section is crucial for ensuring ethical use of your AI counseling assistant.
Mental health is a sensitive domain requiring careful consideration of ethics, limitations, and potential risks. Your AI counseling assistant should always be used as a supplementary tool, never replacing professional care.

In [13]:
print("""
===== MENTAL HEALTH COUNSELING MODEL USAGE GUIDELINES =====

This fine-tuned model is intended to provide supportive responses for mental health concerns.
However, it has important limitations:

1. This model is NOT a replacement for professional mental health services.
2. For crisis situations or immediate risk, please contact emergency services or crisis hotlines:
   - National Suicide Prevention Lifeline: 988 or 1-800-273-8255
   - Crisis Text Line: Text HOME to 741741

Ethical usage:
- Always disclose that responses are AI-generated
- Monitor responses for harmful or inappropriate content
- Use only as a supplementary tool alongside professional care
- Handle user data with appropriate privacy and confidentiality measures

Remember: The model generates responses based on patterns in its training data
and may not always be accurate, appropriate, or helpful for all situations.
""")


===== MENTAL HEALTH COUNSELING MODEL USAGE GUIDELINES =====

This fine-tuned model is intended to provide supportive responses for mental health concerns.
However, it has important limitations:

1. This model is NOT a replacement for professional mental health services.
2. For crisis situations or immediate risk, please contact emergency services or crisis hotlines:
   - National Suicide Prevention Lifeline: 988 or 1-800-273-8255
   - Crisis Text Line: Text HOME to 741741

Ethical usage:
- Always disclose that responses are AI-generated
- Monitor responses for harmful or inappropriate content
- Use only as a supplementary tool alongside professional care
- Handle user data with appropriate privacy and confidentiality measures

Remember: The model generates responses based on patterns in its training data
and may not always be accurate, appropriate, or helpful for all situations.

